### Prefix Sums

In [1]:
x = [1, 4, 3, 5, 6, 7, 0, 1]

rdd = sc.parallelize(x, 4).cache()

def f(iterator):
    yield sum(iterator)

sums = rdd.mapPartitions(f).collect()

print(sums)

for i in range(1, len(sums)):
    sums[i] += sums[i-1]

print(sums)

def g(index, iterator):
    global sums
    if index == 0:
        s = 0
    else:
        s = sums[index-1]
    for i in iterator:
        s += i
        yield s

prefix_sums = rdd.mapPartitionsWithIndex(g)
print(prefix_sums.collect())

[5, 8, 13, 1]
[5, 13, 26, 27]
[1, 5, 8, 13, 19, 26, 26, 27]


### Monotocity checking

In [17]:
x = [1, 3, 4, 5, 4, 8, 10, 14, 16, 20, 21, 24, 24, 26, 27, 30]

rdd = sc.parallelize(x, 4).cache()

def f(it):
    first = next(it)
    last = first
    increasing = True
    print(it, first,last)
    for i in it:
        if i < last:
            increasing = False
        last = i
    yield increasing, first, last

results = rdd.mapPartitions(f).collect()
print(results)

increasing = True
if results[0][0] == False:
    increasing = False
else:
    for i in range(1, len(results)):
        if results[i][0] == False or results[i][1] < results[i-1][2]:
            increasing = False
if increasing:
    print("Monotone")
else:
    print("Not monotone")



[(True, 1, 5), (True, 4, 14), (True, 16, 24), (True, 24, 30)]


<itertools.chain object at 0x7fd0e108edc0> 4 4
<itertools.chain object at 0x7fd0e108edc0> 1 1
<itertools.chain object at 0x7fd0e108edc0> 16 16
<itertools.chain object at 0x7fd0e108edc0> 24 24


[(True, 1, 5), (True, 4, 14), (True, 16, 24), (True, 24, 30)]
Not monotone


### Maximum Subarray Problem

In [11]:
# Classical divide and conquer algorithm

A = [-3, 2, 1, -4, 5, 2, -1, 3, -1]

def MaxSubarray(A, p, r):
    if p == r:
        return A[p]
    q = (p+r)//2
    M1 = MaxSubarray(A, p, q)
    M2 = MaxSubarray(A, q+1, r)
    Lm = -float('inf')
    Rm = Lm
    V = 0
    for i in range(q, p-1, -1):
        V += A[i]
        if V > Lm:
            Lm = V
    V = 0
    for i in range(q+1, r+1):
        V += A[i]
        if V > Rm:
            Rm = V
    return max(M1, M2, Lm+Rm)

print(MaxSubarray(A, 0, len(A)-1))

9


In [14]:
# Linear-time algorithm
# Written in a way so that we can call it for each partition

def linear_time(it):
    Vmax = -float('inf')
    V = 0
    for Ai in it:
        V += Ai
        if V > Vmax:
            Vmax = V
        if V < 0:
            V = 0
    yield Vmax
    
print(next(linear_time(A)))

9


In [17]:
# The Spark algorithm:

def compute_sum(it):
    yield sum(it)

def compute_LmRm(index, it):
    Rm = -float('inf')
    L = sums[index]
    Lm = L
    R = 0
    for Ai in it:
        L -= Ai
        R += Ai
        if L > Lm:
            Lm = L
        if R > Rm:
            Rm = R
    yield (Lm, Rm)

num_partitions = 4
rdd = sc.parallelize(A, num_partitions).cache()
sums = rdd.mapPartitions(compute_sum).collect()
print(sums)
LmRms = rdd.mapPartitionsWithIndex(compute_LmRm).collect()
print(LmRms)
best = max(rdd.mapPartitions(linear_time).collect())

for i in range(num_partitions-1):
    for j in range(i+1, num_partitions):
        x = LmRms[i][0] + sum(sums[i+1:j]) + LmRms[j][1]
        if x > best:
            best = x

print(best)

[-1, -3, 7, 1]
[(2, -1), (0, 1), (2, 7), (2, 2)]
9


In [8]:
def f(index, it):
    for i in it:
        yield i+index

A = sc.parallelize(range(100), 4).mapPartitionsWithIndex(f)
print(A.take(100))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [47]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)
#print(rdd.glom().collect())

def f(index, it):
    label = "="
    for i in it:
        if i[0] == i[1]:
            continue
        if i[0]>i[1]:
            label = ">"
            break
        else:
            label = "<"
            break
    yield index, label

# FILL IN YOUR CODE HERE
res = rdd.mapPartitionsWithIndex(f).collect()
#print(res)

final_res = "="
for i in range(len(res)):
    if res[i][1] == "=":
        continue
    elif res[i][1]!= "=":
        final_res = res[i][1]
        break

print("",final_res)

[(0, '='), (1, '<'), (2, '='), (3, '=')]
x is < than y


In [38]:
x = [1, 3, 4, 5, 4, 8, 10, 14, 16, 20, 21, 24, 24, 26, 27, 30]

rdd = sc.parallelize(x, 4).cache()

def f(it):
    first = next(it)
    last = first
    increasing = True
    print(it, first,last)
    for i in it:
        if i < last:
            increasing = False
        last = i
    yield increasing, first, last

results = rdd.mapPartitions(f).collect()
print(results)

increasing = True
if results[0][0] == False:
    increasing = False
else:
    for i in range(1, len(results)):
        if results[i][0] == False or results[i][1] < results[i-1][2]:
            increasing = False
if increasing:
    print("Monotone")
else:
    print("Not monotone")




[(True, 1, 5), (True, 4, 14), (True, 16, 24), (True, 24, 30)]
Not monotone


<itertools.chain object at 0x7fd0e108ed00> 24 24
<itertools.chain object at 0x7fd0e108ed00> 4 4
<itertools.chain object at 0x7fd0e108ed00> 1 1
<itertools.chain object at 0x7fd0e108ed00> 16 16


In [18]:
x = [1, 3, 4, 5, 4, 8, 10, 14, 16, 20, 21, 24, 24, 26, 27, 30]

rdd = sc.parallelize(x, 4).cache()

def f(it):
    first = next(it)
    last = first
    increasing = True
    print(it, first,last)
    for i in it:
        if i < last:
            increasing = False
        last = i
    yield increasing, first, last

results = rdd.mapPartitions(f).collect()
print(results)

increasing = True
if results[0][0] == False:
    increasing = False
else:
    for i in range(1, len(results)):
        if results[i][0] == False or results[i][1] < results[i-1][2]:
            increasing = False
if increasing:
    print("Monotone")
else:
    print("Not monotone")




[(True, 1, 5), (True, 4, 14), (True, 16, 24), (True, 24, 30)]
Not monotone


<itertools.chain object at 0x7fd0e108ed00> 24 24
<itertools.chain object at 0x7fd0e108ed00> 16 16
<itertools.chain object at 0x7fd0e108ed00> 1 1
<itertools.chain object at 0x7fd0e108ed00> 4 4


In [46]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)





# FILL IN YOUR CODE HERE
res = rdd.mapPartitionsWithIndex(f).collect()

label = "="
def f(index, it):
    for i in it:
        if i[0] == i[1]:
            continue
        if i[0]>i[1]:
            label = ">"
            break
        else:
            label = "<"
            break
    yield index, label
print(res)

final_res = "="
for i in range(len(res)):
    if res[i][1] == "=":
        continue
    elif res[i][1]!= "=":
        final_res = res[i][1]
        break

print("x is",final_res, "than y")

IndentationError: expected an indented block (4074154813.py, line 20)

In [13]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)


def f(index, it):
    label = '='
    for i in it:
        if i[0] == i[1]:
            continue
        if i[0]>i[1]:
            label = ">"
            break
        else:
            label = "<"
            break
    yield index, label

# FILL IN YOUR CODE HERE
res = rdd.mapPartitionsWithIndex(f).collect()
print(res)

final_res = "="
for i in range(len(res)):
    if res[i][1] == "=":
        continue
    elif res[i][1]!= "=":
        final_res = res[i][1]
        break

print("lexicographical order of x and y is ",final_res)

[(0, '='), (1, '<'), (2, '='), (3, '=')]
lexicographical order of x and y is  <


In [16]:
numPartitions = 10

path_to_file = "adj_noun_pairs.txt"
lines = sc.textFile(path_to_file, numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

# FILL IN YOUR CODE HERE
def f(index, it):
    adj = ""
    for i in it:
        if i[1] == "unification":
            adj = i[0]
            break
    yield index, adj
    
res = pairs.mapPartitionsWithIndex(f).collect()    
print(res)
final_res = ""
for i in range(len(res)):
    if res[i][1] != "":
        final_res = res[i][1]
        break

print("first adj is ",final_res)

23/04/04 11:52:57 WARN BlockManager: Task 42 already completed, not releasing lock for rdd_43_7
23/04/04 11:52:57 WARN BlockManager: Task 43 already completed, not releasing lock for rdd_43_8
23/04/04 11:52:57 WARN BlockManager: Task 44 already completed, not releasing lock for rdd_43_9
[(0, ''), (1, ''), (2, ''), (3, ''), (4, 'several'), (5, ''), (6, 'national'), (7, 'Official'), (8, 'large-scale'), (9, 'italian')]
first adj is  several


In [14]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)



def f(index, it):
    label = "="
    for i in it:
        if i[0] == i[1]:
            continue
        if i[0]>i[1]:
            label = ">"
            break
        else:
            label = "<"
            break
    yield index, label

# FILL IN YOUR CODE HERE
res = rdd.mapPartitionsWithIndex(f).collect()
#print(res)

final_res = "="
for i in range(len(res)):
    if res[i][1] == "=":
        continue
    elif res[i][1]!= "=":
        final_res = res[i][1]
        break

print("lexicographical order of x and y is ",final_res)

SyntaxError: invalid non-printable character U+00A0 (503954169.py, line 10)

In [17]:
def f(index, it):
    for i in it:
        yield i+index

A = sc.parallelize(range(100), 4).mapPartitionsWithIndex(f)
print(A.take(100))

SyntaxError: invalid non-printable character U+00A0 (1110202099.py, line 2)